# Lab 1.2.5: Profiling Exercise - SOLUTIONS

**Module:** 1.2 - Python for AI/ML  

This notebook contains solutions to all exercises from the Profiling Exercise Lab.

---

In [ ]:
import numpy as np
import time

print("Solutions Notebook for Profiling Exercise")
print("=" * 50)

---

## Exercise: Optimized K-Nearest Neighbors

**Task:** Optimize the slow KNN implementation.

In [ ]:
# Original slow version (for reference)
def knn_slow(query_points, reference_points, k):
    """Slow KNN with nested loops."""
    n_queries = len(query_points)
    n_ref = len(reference_points)
    
    all_indices = []
    all_distances = []
    
    for i in range(n_queries):
        distances = []
        for j in range(n_ref):
            diff = query_points[i] - reference_points[j]
            dist = np.sqrt(np.sum(diff ** 2))
            distances.append(dist)
        
        distances = np.array(distances)
        sorted_indices = np.argsort(distances)
        
        all_indices.append(sorted_indices[:k])
        all_distances.append(distances[sorted_indices[:k]])
    
    return np.array(all_indices), np.array(all_distances)

In [ ]:
# SOLUTION: Optimized KNN
def knn_fast(query_points, reference_points, k):
    """
    Optimized k-nearest neighbors using vectorization.
    
    Key optimizations:
    1. Use ||a-b||^2 = ||a||^2 + ||b||^2 - 2*a·b trick
    2. Compute all pairwise distances at once
    3. Use np.argpartition instead of full sort (O(n) vs O(n log n))
    
    Args:
        query_points: (n_queries, dims)
        reference_points: (n_reference, dims)
        k: number of neighbors
    
    Returns:
        indices: (n_queries, k) - indices of k nearest neighbors
        distances: (n_queries, k) - distances to k nearest neighbors
    """
    # Step 1: Compute squared norms
    query_sq = np.sum(query_points ** 2, axis=1, keepdims=True)  # (n_q, 1)
    ref_sq = np.sum(reference_points ** 2, axis=1)  # (n_r,)
    
    # Step 2: Compute all pairwise squared distances
    # ||q - r||^2 = ||q||^2 + ||r||^2 - 2*q·r
    # Broadcasting: (n_q, 1) + (n_r,) - 2 * (n_q, n_r) -> (n_q, n_r)
    sq_distances = query_sq + ref_sq - 2 * (query_points @ reference_points.T)
    
    # Handle numerical errors (small negative values from floating point)
    sq_distances = np.maximum(sq_distances, 0)
    
    # Step 3: Find top-k using argpartition (faster than full sort!)
    # argpartition puts the k smallest elements in the first k positions
    # but doesn't sort them
    indices = np.argpartition(sq_distances, k, axis=1)[:, :k]
    
    # Step 4: Get the actual squared distances for these indices
    # Use advanced indexing
    row_indices = np.arange(len(query_points))[:, np.newaxis]
    top_k_sq_distances = sq_distances[row_indices, indices]
    
    # Step 5: Sort within the k (argpartition doesn't guarantee order)
    sorted_within_k = np.argsort(top_k_sq_distances, axis=1)
    
    # Reorder indices and distances
    final_indices = np.take_along_axis(indices, sorted_within_k, axis=1)
    final_sq_distances = np.take_along_axis(top_k_sq_distances, sorted_within_k, axis=1)
    
    # Convert squared distances to distances
    final_distances = np.sqrt(final_sq_distances)
    
    return final_indices, final_distances

print("Optimized KNN function defined!")

In [ ]:
# Test and verify correctness
np.random.seed(42)
queries = np.random.randn(50, 32).astype(np.float32)
references = np.random.randn(200, 32).astype(np.float32)
k = 5

# Run both versions
start = time.perf_counter()
indices_slow, distances_slow = knn_slow(queries, references, k)
slow_time = time.perf_counter() - start

start = time.perf_counter()
indices_fast, distances_fast = knn_fast(queries, references, k)
fast_time = time.perf_counter() - start

# Verify correctness
indices_match = np.allclose(indices_slow, indices_fast)
distances_match = np.allclose(distances_slow, distances_fast, rtol=1e-5)

print("Correctness check:")
print(f"  Indices match: {indices_match}")
print(f"  Distances match: {distances_match}")

print(f"\nPerformance comparison:")
print(f"  Slow version: {slow_time*1000:.2f} ms")
print(f"  Fast version: {fast_time*1000:.2f} ms")
print(f"  Speedup: {slow_time/fast_time:.1f}x")

In [ ]:
# Scale test
print("\nScale test (larger data):")
print("-" * 50)

sizes = [(100, 500), (200, 1000), (500, 2000), (1000, 5000)]

for n_queries, n_refs in sizes:
    queries = np.random.randn(n_queries, 64).astype(np.float32)
    references = np.random.randn(n_refs, 64).astype(np.float32)
    k = 10
    
    # Only time fast version for larger sizes
    if n_queries <= 100 and n_refs <= 500:
        start = time.perf_counter()
        _ = knn_slow(queries, references, k)
        slow_time = (time.perf_counter() - start) * 1000
    else:
        slow_time = float('nan')
    
    start = time.perf_counter()
    _ = knn_fast(queries, references, k)
    fast_time = (time.perf_counter() - start) * 1000
    
    if np.isnan(slow_time):
        print(f"  {n_queries:4d} queries, {n_refs:4d} refs: Fast = {fast_time:7.1f} ms (slow would be too slow)")
    else:
        print(f"  {n_queries:4d} queries, {n_refs:4d} refs: Slow = {slow_time:7.1f} ms, Fast = {fast_time:7.1f} ms")

### Alternative: Even Faster with scipy.spatial

In [ ]:
from scipy.spatial import cKDTree

def knn_kdtree(query_points, reference_points, k):
    """
    KNN using KD-Tree (best for low-dimensional data).
    
    Note: KD-Trees are efficient for low dimensions (d < ~20).
    For high-dimensional data, the brute-force vectorized approach may be faster.
    """
    tree = cKDTree(reference_points)
    distances, indices = tree.query(query_points, k=k)
    return indices, distances

# Compare all three methods on low-dimensional data
print("Comparison with KD-Tree (low-dimensional):")
queries = np.random.randn(500, 10).astype(np.float32)  # Low dimensional
references = np.random.randn(2000, 10).astype(np.float32)
k = 10

# Fast vectorized
start = time.perf_counter()
_, _ = knn_fast(queries, references, k)
fast_time = (time.perf_counter() - start) * 1000

# KD-Tree
start = time.perf_counter()
_, _ = knn_kdtree(queries, references, k)
kdtree_time = (time.perf_counter() - start) * 1000

print(f"  Vectorized: {fast_time:.1f} ms")
print(f"  KD-Tree:    {kdtree_time:.1f} ms")

# High-dimensional comparison
print("\nComparison with KD-Tree (high-dimensional):")
queries = np.random.randn(500, 128).astype(np.float32)  # High dimensional
references = np.random.randn(2000, 128).astype(np.float32)

start = time.perf_counter()
_, _ = knn_fast(queries, references, k)
fast_time = (time.perf_counter() - start) * 1000

start = time.perf_counter()
_, _ = knn_kdtree(queries, references, k)
kdtree_time = (time.perf_counter() - start) * 1000

print(f"  Vectorized: {fast_time:.1f} ms")
print(f"  KD-Tree:    {kdtree_time:.1f} ms")
print(f"\n💡 KD-Trees suffer from 'curse of dimensionality' in high-D!")

---

## Bonus: Memory-Efficient KNN for Very Large Datasets

In [ ]:
def knn_chunked(query_points, reference_points, k, chunk_size=1000):
    """
    Memory-efficient KNN that processes in chunks.
    
    Useful when the full distance matrix doesn't fit in memory.
    
    Args:
        query_points: (n_queries, dims)
        reference_points: (n_reference, dims)
        k: number of neighbors
        chunk_size: number of queries to process at once
    
    Returns:
        indices, distances: both (n_queries, k)
    """
    n_queries = len(query_points)
    all_indices = []
    all_distances = []
    
    # Precompute reference squared norms
    ref_sq = np.sum(reference_points ** 2, axis=1)
    
    for start in range(0, n_queries, chunk_size):
        end = min(start + chunk_size, n_queries)
        chunk = query_points[start:end]
        
        # Compute distances for this chunk
        query_sq = np.sum(chunk ** 2, axis=1, keepdims=True)
        sq_distances = query_sq + ref_sq - 2 * (chunk @ reference_points.T)
        sq_distances = np.maximum(sq_distances, 0)
        
        # Find top-k
        indices = np.argpartition(sq_distances, k, axis=1)[:, :k]
        row_idx = np.arange(len(chunk))[:, np.newaxis]
        top_k_sq = sq_distances[row_idx, indices]
        
        # Sort within k
        sorted_within = np.argsort(top_k_sq, axis=1)
        final_indices = np.take_along_axis(indices, sorted_within, axis=1)
        final_distances = np.sqrt(np.take_along_axis(top_k_sq, sorted_within, axis=1))
        
        all_indices.append(final_indices)
        all_distances.append(final_distances)
    
    return np.vstack(all_indices), np.vstack(all_distances)

# Test chunked version
indices_chunked, distances_chunked = knn_chunked(queries, references, k=10, chunk_size=100)
indices_fast, distances_fast = knn_fast(queries, references, k=10)

print("Chunked version matches fast version:")
print(f"  Indices: {np.allclose(indices_chunked, indices_fast)}")
print(f"  Distances: {np.allclose(distances_chunked, distances_fast)}")

---

## Key Optimization Lessons

1. **Vectorize loops** - NumPy operations are 100-1000x faster than Python loops

2. **Use math tricks** - The ||a-b||² = ||a||² + ||b||² - 2a·b trick avoids materializing difference arrays

3. **Choose the right algorithm** - `argpartition` is O(n) vs `argsort` O(n log n) for top-k

4. **Consider memory** - Chunking helps when data doesn't fit in memory

5. **Know your data** - KD-Trees work for low-D, brute-force for high-D

---

**End of Solutions**